## Dergipark.org'tan İlk sayfanın verilerini çekme

In [13]:
# VERİ ÇEKMEK İÇİN GEREKLİ KÜTÜPHANELERİN EKLENMESİ
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from IPython.display import display, HTML

# YAZAR İSİMLERİ
"""
    Verilen kataloglar listesindeki her bir katalog için aşağıdaki işlemleri yaparak sonucu döndürür:
    1. Kategori adını alır. Eğer kategori adı boş ise, tüm satırları virgülle birleştirir.
    2. Kategori adı boş değilse ve satır sayısı 1'den büyükse, kategori adını ve ilk satırı alır, diğer satırları virgülle birleştirir.
    3. Eğer satırlar içinde 'Sayı ', 'Cilt', '(..)' veya '10./' içeren bir ifade varsa, o satıra kadar olan kısmı alır, diğer kısmı atlar.
    4. Son olarak, filtrelenmiş satırları birleştirerek son sonucu oluşturur ve döndürür.
"""
def yazar_isimleri(kataloglar): # "yazar_isimleri" adı altında bir fonksiyon oluşturma
    kategori = [
        " " if len(sayfa_i.split("\n")) <= 1 or sayfa_i.split("\n")[1].count(',') > 1 else sayfa_i.split("\n")[1]
        for sayfa_i in kataloglar
    ]
    
    sonuc = [] # sonuc boş liste oluşturma
    for i, sayfa_i in enumerate(kataloglar):
        split_items = sayfa_i.split("\n") # katalogları "\n" ayırma
        if kategori[i] != " " and len(split_items) > 1: # kategori i. değeri eşit değilse boş değere ve split_item büyükse 1' den
            joined_items = split_items[0] + " , " + " , ".join(split_items[2:]) + "\n" # kategori bölümünü hariç tut
        else:
            joined_items = " , ".join(split_items) + "\n" # kategori bölümünü tut
        sonuc.append(joined_items)

    final_result = [] # final_result boş liste oluşturma
    for i in sonuc: 
        split_result = i.split(" , ") # "split_result" "," ayırma
        filtered_result = [] # filtered_result boş liste oluşturma
        stop_flag = False 
        for item in split_result:
            filtered_result.append(item)
            if 'Sayı ' in item or 'Cilt' in item or (item.startswith(" (") and item.endswith(")")) or ("10." in item and "/" in item and "." in item): # Eğer satırlar içinde 'Sayı ', 'Cilt', '(..)' veya '10./' içeren bir ifade varsa, o satıra kadar olan kısmı alır, diğer kısmı atlar.
                stop_flag = True
                break
        if not stop_flag:
            filtered_result = split_result
        final_result.append(", ".join(filtered_result[1:-1])) # kategoride 1. index ve  -1. index arasındaki yazar isimlerini alma
    return final_result

# DERGİ İSİMLERİ EKLEME
def dergi_isimleri(kataloglar): # "dergi_isimleri" adı altında bir fonksiyon oluşturma
    sonuc2 = [] # sonuc2 adında bir boş liste oluşturma
    for i in kataloglar: 
        deger = " "  # dergi değeri başlangıçta " " olarak ayarla
        j_list = i.split(",") # virgül ile j_list adı altın liste oluşturma
        for index, j in enumerate(j_list):
            if (len(j) == 7 and j.startswith(" (") and j.endswith(")")): # eğer "(2020)" ise
                if index + 1 < len(j_list):  # tarihten bir sonraki index değerini al
                    deger = j_list[index + 1].split("\n")[0]
                break
            if "Cilt" in j: # eğer "Cilt" ile başlıyorsa 
                if index - 1 >= 0:  
                    deger = j_list[index - 1] # "Cilt" değerinden önceki index değerini al
                break
            if "Sayı" in j:
                if index - 1 >= 0:  # eğer "Sayı" ile başlıyorsa 
                    deger = j_list[index - 1]  # "Sayı" değerinden önceki index değerini al
                break
            if j.replace(" ", "").replace("-", "").isdigit(): # eğer "1-20" sayfa sayısı ile başlıyorsa 
                if index - 1 >= 0: 
                    deger = j_list[index - 1]  # "1-20" sayfa sayısı değerinden önceki index değerini al
                break
        sonuc2.append(deger)
    return sonuc2

# CİLT SAYILARI EKLEME
def cilt_degerlerini_al(kataloglar): # "cilt_degerlerini_ adı altında fonksiyon oluşturma
    sonuc3 = [] # sonuc3 boş liste oluşturma
    for sayfa in kataloglar:
        satirlar = sayfa.split("\n") # makale listesi \n den ayrılıyorsa
        cilt_degeri = " "  # cilt_değeri başlangıçta " " olarak ayarla
        for satir in satirlar: 
            if "Cilt" in satir: # eğer liste içinde "Cilt" varsa
                # 'Cilt' kelimesinden sonra gelen değeri al
                parca = satir.split("Cilt")[1].split(",")[0].strip() # o zaman Cilt değerinin 1. indexinin virgül ile listesini oluşturup 0. indexi alma
                if parca.isdigit():  # Eğer parça bir sayıysa
                    cilt_degeri = parca # cilt_degerine parca atama
                break
        sonuc3.append(cilt_degeri)
    return sonuc3

# DERGİ SAYISI EKLEME
def sayi_degerlerini_al(kataloglar): # sayi_degerlerini_al adı altında fonksiyon oluşturma
    sonuc4 = [] # sonuc4 boş liste oluşturma
    for sayfa in kataloglar:
        satirlar = sayfa.split("\n") # makale listesi \n den ayrılıyorsa liste oluştur
        sayi_degeri = " "  # sayi_degeri başlangıçta " " olarak ayarla
        for satir in satirlar:
            if "Sayı" in satir: # eğer "Sayı" içeriyorsa
                parca = satir.split("Sayı")[1].split(",")[0].strip()  # 'Sayı' kelimesinden sonra gelen değeri al
                if parca.isdigit():  # Eğer parca bir sayıysa
                    sayi_degeri = parca # sayi_degeri parca atama
                break
        sonuc4.append(sayi_degeri)
    return sonuc4

# TARİH EKLEME
def tarih_bilgisi_ekle(kataloglar):  # tarih_bilgisi_ekle adı altında fonksiyon oluşturma
    sonuc5 = [] # sonuc5 boş liste oluşturma
    for i in kataloglar:
        deger = " " # dergi değeri başlangıçta " " olarak ayarla
        j_list = i.split(",") # virgül ile j_list adı altında liste oluşturma
        for index, j in enumerate(j_list):
            if len(j) == 7 and j.startswith(" (") and j.endswith(")"): # eğer  "("  başlıyor ve ")" bitiyorsa ve karakter uzuntuluğu 7 ise
                deger = j.split("\n")[0][2:6]  # tarih değerlerini al
                break 
            if j.replace(" ", "").replace("-", "").isdigit(): # eğer "1-20" sayfa numarası varsa
                if index + 1 >= 0:
                    deger = j_list[index] # "1-20" sayfa numarasından 1 önceki index değerini al
                break
            if "10." in j and "/" in j and "." in j: # eğer "DOI" uzantısı varsa
                if index - 1 >= 0:
                    deger = j_list[index - 2] # "DOI" uzantısından  1 önceki index değerini al
                break
        sonuc5.append(deger)
    return sonuc5

# SAYFA EKLEME
def sayfa_sayilari_ekle(kataloglar): # sayfa_sayilari_ekle adı altında fonksiyon oluşturma
    sonuc6 = [] # sonuc6 boş liste oluşturma
    for i in kataloglar:
        deger2 = " "  # deger2 başlangıçta " " olarak ayarla
        for j in i.split(","): # virgül ile j adı altında liste oluşturma
            if " " in j and " - " in j: # eğer j listesi içinde " " ve " - "  varsa
                parca = j.split("\n")[0].strip()
                if parca.replace(" ", "").replace("-", "").isdigit(): # "" ve  -"  dönüştür
                    deger2 = parca # sayfa numaralarını "deger2" ye atama
                break  
        sonuc6.append(deger2)
    return sonuc6

# DOI EKLEME 
def doi(kataloglar): # doi adı altında fonksiyon oluşturma
    sonuc7 = [] # sonuc7 boş liste oluşturma
    for i in kataloglar:
        eklenecek_deger = " " # eklenecek_deger başlangıçta " " olarak ayarla
        for j in i.split(","):
            if "10." in j and "/" in j and "." in j: # eğer j listesi içinde "10." ve "/"  ve "." varsa
                eklenecek_deger = j.split("\n")[0] # "DOI" değerlernin "eklenecek_deger" ata
                break  
        sonuc7.append(eklenecek_deger)
    return sonuc7

def make_clickable(url): # make_clickable  altında fonksiyon oluşturma
     return f'<a href="https://doi.org/{url.strip()}" target="_blank">{url.strip()}</a>' # HTML bağlantı linki ve köprü oluşturma

# Anahtar kelime gir
anahtar_kelime = input("Anahtar kelime giriniz: ")

# Web tarayıcısını başlat
driver = webdriver.Chrome()


kataloglar = []

 # URL oluşturma
driver.get(f"https://dergipark.org.tr/tr/search?q={anahtar_kelime}&section=articles")

# Sayfa yüklenene kadar bekleyin
WebDriverWait(driver, 60).until(EC.invisibility_of_element((By.ID, "loadingOverlay")))

# Kritere uygun makaleleri listeleyen elementleri bulmak için bekleme yap
makaleler = WebDriverWait(driver, 20).until(
    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[class="card article-card dp-card-outline"]'))
)

kataloglar = []
for makale in makaleler:
    katalog = makale.find_element(By.CSS_SELECTOR, '[class="card-body"]').text
    kataloglar.append(katalog)
    

# Sözlük oluşturma
veri = {
    "Başlık": [baslik_i.split("\n")[0] for baslik_i in kataloglar], # Başlık getirme
    "Kategori": [" " if len(sayfa_i.split("\n")) <= 1 or sayfa_i.split("\n")[1].count(',') > 1 else sayfa_i.split("\n")[1] for sayfa_i in kataloglar], # Kategori getirme
    "Sorumlu Yazarlar": yazar_isimleri(kataloglar), # Yazar isimleri getirme
    "Dergi İsmi": dergi_isimleri(kataloglar), # Dergi isimlerini getirme
    "Cilt Sayısı": cilt_degerlerini_al(kataloglar), # Cilt sayılarını getirme
    "Sayı": sayi_degerlerini_al(kataloglar), # Sayıları getirme
    "Tarih": tarih_bilgisi_ekle(kataloglar), # Tarih bilgilerini getirme
    "Sayfa": sayfa_sayilari_ekle(kataloglar), # Sayfa numarlarını getirme
    "DOI": doi(kataloglar), # DOI getirme
    "Özet" : [",".join(j[3:-1]) for j in [i.split("\n") for i in kataloglar]] # Özet bilgilerini getirme
}

# DataFrame oluşturma
makale_tablosu = pd.DataFrame(veri) 

# "make_clickable" kütüphanesindeki URL fonksiyonunu "makale_tablosu2["DOI"]" sütununa uygulama
makale_tablosu["DOI"] = makale_tablosu["DOI"].apply(make_clickable)

# "makale_tablosu2" deki "DOI"leri HTML koduna dönüştürme
display(HTML(makale_tablosu.to_html(escape=False)))

# "makale_tablosu2" yi Excel formatına dönüştürme
makale_tablosu.to_excel("makale_listesi.xlsx", index=False)

# Tarayıcıyı kapat
driver.quit()

Anahtar kelime giriniz:  yapay zeka


,Başlık,Kategori,Sorumlu Yazarlar,Dergi İsmi,Cilt Sayısı,Sayı,Tarih,Sayfa,DOI,Özet
0,YAPAY ZEKA,,Arş. Gör. Harun Pirim,Yaşar Üniversitesi E-Dergisi,1,1,2006,81 - 93,,
1,Sinemanın Yapay Zekâya Bakışı,Diğer,İbrahim Acar,TRT Akademi,6,13,2021,918 - 929,10.37679/trta.1002528,"Film eleştirmeni Burak Göral tarafından ""Yapay Zeka"" sayımıza özel yapay zeka konulu filmler üzerine yazılmış genel bir değerlendirme yazısıdır."
2,Yapay Zekâ ve Stratejik Yönetim,Editoryal,Necla Öykü İyigün,TRT Akademi,6,13,2021,675 - 679,10.37679/trta.1002518,"TRT Akademi Dergisinin Eylül 2021'de yayınlanan ""Yapay Zekâ"" temalı 13. sayısı için kaleme alınmış editör yazısıdır."
3,Dilbilim ve Yapay Zeka,Derleme,Wolf König,Dilbilim Araştırmaları Dergisi,5,,1994,219 - 235,,"Dilbilim ile yapay zekâ arasında son 10-15 yıldan bu yana sıkı bir ilişki vardır. Bu ilişki, 30 yıl,""bilgisayar dilbilimi ve yapay zekâ"" diye adlandırılan akademik anadallann oluşmasını görebiliyoruz,. Bu gelişme, henüz 10 yıldır sürmektedir. Bilişimin, yani, bilgisayar biliminin bir bölümü olan yapay"
4,Eğitimde Yapay Zeka ve Uygulamaları,Derleme,Kürşat Arslan,Batı Anadolu Eğitim Bilimleri Dergisi,11,1,2020,71 - 88,,"İlk olarak 1956 yılında Dortmund konferansında John McCarthy tarafından dile getirilen yapay zeka,çapında birçok ülke ve sayısız şirket yapay zeka konusunda farklı uygulama alanlarına çok miktarda,, yapay zeka konusunda dikkat çeken yatırım alanlarından biridir. Yapay zekanın tanımı arasında,sağlama, farklı insan dili ve deneyimlerine adapte olma gibi özellikleriyle yapay zeka, başta eğitimde,almaktadır. Aslında bugün yapay zeka çoktan sınıf içine girmiş, öğrencilerin, öğretmenlerin ya da ailelerin"
5,Acil Serviste Yapay Zeka Kullanımı,Tartışma,"Zamir Kemal Ertürk, Bahadır Ertürk",Aksaray Üniversitesi Tıp Bilimleri Dergisi,2,2,2021,39 - 40,,"Yapay zeka, bir bilgisayarın veya bilgisayar kontrollü bir makinenin akıl yürütme, anlamlandırma,getirme yeteneğidir. 80’ler ve 90’larda yapay zeka üzerine bir çok çalışma yapıldı ancak toplumların,aktif olarak kullanımı 2000’li yıllarda olabildi. Yapay zeka uygulamalarının ve derin öğrenme,algoritmalarının tıp alanında özellikle acil servislerde kullanımı hızla artış göstermektedir. Bu yazıda yapay"
6,Robotların Yükselişi: Yapay Zeka ve İşsiz Bir Gelecek Tehlikesi,Kitap İncelemesi,Sena Nadide Kaya,Novus Orbis: Siyaset Bilimi ve Uluslararası İlişkiler Dergisi,4,1,2022,115 - 119,,"Martin Ford, Robotların Yükselişi: Yapay Zeka ve İşsiz Bir Gelecek Tehlikesi, İstanbul: Kronik Yayınları, ss. 336"
7,YAPAY ZEKANIN ETİK GERÇEKLİĞİ,Tez Özeti,Gizem Öztürk Dilek,Ankara Uluslararası Sosyal Bilimler Dergisi,2,4,2019,47 - 59,,"Yapay zekâ, gelecekteki yeri açısından etik söyleme ihtiyaç duymaktadır. Özelikle bilgisayarın ve,yapay zekânın insan hayatında yer edinmesi günümüzde düşünme ve iş becerilerini kısıtlayacağına,; gelecekte ise insan ırkı için sorun yaratabileceği düşüncesi ile yapay zekânın etik söyleme ihtiyacı,oluşturmak mümkün görünmektedir. Bu anlamda günümüzün etik anlayışı olan uygulamalı etik, yapay zekâya, çözüm önerisi sunmayı amaçlamaktadır. Yapay zekânın etik söylemini oluşturmayı amaçlayan"
8,Yapay Zekanın Toplum Üzerindeki Etkisi ve Yapay Zeka (A.I.) Filmi Bağlamında Yapay Zekaya Bakış,Derleme,İpek Sucu,Uluslararası Ders Kitapları ve Eğitim Materyalleri Dergisi,2,2,2019,203 - 215,,"buluşların anahtarı olmuştur. “Nasıl ki ilk çağlarda en önemli buluş ateşin keşfi ise, yapay zekâ da,olduğumuz teknoloji çağının en önemli buluşlarından biri de “Yapay Zekâ” teknolojisidir. Yapay zeka,kavramından son yıllarda sıkça bahsedilmektedir. Aslında yapay zeka kavramı bu kadar yeni bir kavram,değil, aksine yapay zekanın ortaya çıkışı çok eski dönemlere dayanmaktadır. Teknolojik gelişmelere,bağlı olarak çeşitli yapay zeka programları oluşturulmuş ve robotlar yapılmaya başlanmıştır. Bu"
9,YAPAY ZEKA ÇAĞINDA DİPLOMASİ,Araştırma Makalesi,Sefa Can Yılmazel,Türk Dünyası Araştırmaları,132,260,2022,91 - 112,10.55773/tda.1153159,"doğrultuda, günümüz dün

## Dergipark.org'tan İlk 10 sayfanın verilerini çekme

In [12]:
# VERİ ÇEKMEK İÇİN GEREKLİ KÜTÜPHANELERİN EKLENMESİ
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from IPython.display import display, HTML

# YAZAR İSİMLERİ
"""
    Verilen kataloglar listesindeki her bir katalog için aşağıdaki işlemleri yaparak sonucu döndürür:
    1. Kategori adını alır. Eğer kategori adı boş ise, tüm satırları virgülle birleştirir.
    2. Kategori adı boş değilse ve satır sayısı 1'den büyükse, kategori adını ve ilk satırı alır, diğer satırları virgülle birleştirir.
    3. Eğer satırlar içinde 'Sayı ', 'Cilt', '(..)' veya '10./' içeren bir ifade varsa, o satıra kadar olan kısmı alır, diğer kısmı atlar.
    4. Son olarak, filtrelenmiş satırları birleştirerek son sonucu oluşturur ve döndürür.
"""
def yazar_isimleri(kataloglar): # "yazar_isimleri" adı altında bir fonksiyon oluşturma
    kategori = [
        " " if len(sayfa_i.split("\n")) <= 1 or sayfa_i.split("\n")[1].count(',') > 1 else sayfa_i.split("\n")[1]
        for sayfa_i in kataloglar
    ]
    
    sonuc = [] # sonuc boş liste oluşturma
    for i, sayfa_i in enumerate(kataloglar):
        split_items = sayfa_i.split("\n") # katalogları "\n" ayırma
        if kategori[i] != " " and len(split_items) > 1: # kategori i. değeri eşit değilse boş değere ve split_item büyükse 1' den
            joined_items = split_items[0] + " , " + " , ".join(split_items[2:]) + "\n" # kategori bölümünü hariç tut
        else:
            joined_items = " , ".join(split_items) + "\n" # kategori bölümünü tut
        sonuc.append(joined_items)

    final_result = [] # final_result boş liste oluşturma
    for i in sonuc: 
        split_result = i.split(" , ") # "split_result" "," ayırma
        filtered_result = [] # filtered_result boş liste oluşturma
        stop_flag = False 
        for item in split_result:
            filtered_result.append(item)
            if 'Sayı ' in item or 'Cilt' in item or (item.startswith(" (") and item.endswith(")")) or ("10." in item and "/" in item and "." in item): # Eğer satırlar içinde 'Sayı ', 'Cilt', '(..)' veya '10./' içeren bir ifade varsa, o satıra kadar olan kısmı alır, diğer kısmı atlar.
                stop_flag = True
                break
        if not stop_flag:
            filtered_result = split_result
        final_result.append(", ".join(filtered_result[1:-1])) # kategoride 1. index ve  -1. index arasındaki yazar isimlerini alma
    return final_result

# DERGİ İSİMLERİ EKLEME
def dergi_isimleri(kataloglar): # "dergi_isimleri" adı altında bir fonksiyon oluşturma
    sonuc2 = [] # sonuc2 adında bir boş liste oluşturma
    for i in kataloglar: 
        deger = " "  # dergi değeri başlangıçta " " olarak ayarla
        j_list = i.split(",") # virgül ile j_list adı altın liste oluşturma
        for index, j in enumerate(j_list):
            if (len(j) == 7 and j.startswith(" (") and j.endswith(")")): # eğer "(2020)" ise
                if index + 1 < len(j_list):  # tarihten bir sonraki index değerini al
                    deger = j_list[index + 1].split("\n")[0]
                break
            if "Cilt" in j: # eğer "Cilt" ile başlıyorsa 
                if index - 1 >= 0:  
                    deger = j_list[index - 1] # "Cilt" değerinden önceki index değerini al
                break
            if "Sayı" in j:
                if index - 1 >= 0:  # eğer "Sayı" ile başlıyorsa 
                    deger = j_list[index - 1]  # "Sayı" değerinden önceki index değerini al
                break
            if j.replace(" ", "").replace("-", "").isdigit(): # eğer "1-20" sayfa sayısı ile başlıyorsa 
                if index - 1 >= 0: 
                    deger = j_list[index - 1]  # "1-20" sayfa sayısı değerinden önceki index değerini al
                break
        sonuc2.append(deger)
    return sonuc2

# CİLT SAYILARI EKLEME
def cilt_degerlerini_al(kataloglar): # "cilt_degerlerini_ adı altında fonksiyon oluşturma
    sonuc3 = [] # sonuc3 boş liste oluşturma
    for sayfa in kataloglar:
        satirlar = sayfa.split("\n") # makale listesi \n den ayrılıyorsa
        cilt_degeri = " "  # cilt_değeri başlangıçta " " olarak ayarla
        for satir in satirlar: 
            if "Cilt" in satir: # eğer liste içinde "Cilt" varsa
                # 'Cilt' kelimesinden sonra gelen değeri al
                parca = satir.split("Cilt")[1].split(",")[0].strip() # o zaman Cilt değerinin 1. indexinin virgül ile listesini oluşturup 0. indexi alma
                if parca.isdigit():  # Eğer parça bir sayıysa
                    cilt_degeri = parca # cilt_degerine parca atama
                break
        sonuc3.append(cilt_degeri)
    return sonuc3

# DERGİ SAYISI EKLEME
def sayi_degerlerini_al(kataloglar): # sayi_degerlerini_al adı altında fonksiyon oluşturma
    sonuc4 = [] # sonuc4 boş liste oluşturma
    for sayfa in kataloglar:
        satirlar = sayfa.split("\n") # makale listesi \n den ayrılıyorsa liste oluştur
        sayi_degeri = " "  # sayi_degeri başlangıçta " " olarak ayarla
        for satir in satirlar:
            if "Sayı" in satir: # eğer "Sayı" içeriyorsa
                parca = satir.split("Sayı")[1].split(",")[0].strip()  # 'Sayı' kelimesinden sonra gelen değeri al
                if parca.isdigit():  # Eğer parca bir sayıysa
                    sayi_degeri = parca # sayi_degeri parca atama
                break
        sonuc4.append(sayi_degeri)
    return sonuc4

# TARİH EKLEME
def tarih_bilgisi_ekle(kataloglar):  # tarih_bilgisi_ekle adı altında fonksiyon oluşturma
    sonuc5 = [] # sonuc5 boş liste oluşturma
    for i in kataloglar:
        deger = " " # dergi değeri başlangıçta " " olarak ayarla
        j_list = i.split(",") # virgül ile j_list adı altında liste oluşturma
        for index, j in enumerate(j_list):
            if len(j) == 7 and j.startswith(" (") and j.endswith(")"): # eğer  "("  başlıyor ve ")" bitiyorsa ve karakter uzuntuluğu 7 ise
                deger = j.split("\n")[0][2:6]  # tarih değerlerini al
                break 
            if j.replace(" ", "").replace("-", "").isdigit(): # eğer "1-20" sayfa numarası varsa
                if index + 1 >= 0:
                    deger = j_list[index] # "1-20" sayfa numarasından 1 önceki index değerini al
                break
            if "10." in j and "/" in j and "." in j: # eğer "DOI" uzantısı varsa
                if index - 1 >= 0:
                    deger = j_list[index - 2] # "DOI" uzantısından  1 önceki index değerini al
                break
        sonuc5.append(deger)
    return sonuc5

# SAYFA EKLEME
def sayfa_sayilari_ekle(kataloglar): # sayfa_sayilari_ekle adı altında fonksiyon oluşturma
    sonuc6 = [] # sonuc6 boş liste oluşturma
    for i in kataloglar:
        deger2 = " "  # deger2 başlangıçta " " olarak ayarla
        for j in i.split(","): # virgül ile j adı altında liste oluşturma
            if " " in j and " - " in j: # eğer j listesi içinde " " ve " - "  varsa
                parca = j.split("\n")[0].strip()
                if parca.replace(" ", "").replace("-", "").isdigit(): # "" ve  -"  dönüştür
                    deger2 = parca # sayfa numaralarını "deger2" ye atama
                break  
        sonuc6.append(deger2)
    return sonuc6

# DOI EKLEME 
def doi(kataloglar): # doi adı altında fonksiyon oluşturma
    sonuc7 = [] # sonuc7 boş liste oluşturma
    for i in kataloglar:
        eklenecek_deger = " " # eklenecek_deger başlangıçta " " olarak ayarla
        for j in i.split(","):
            if "10." in j and "/" in j and "." in j: # eğer j listesi içinde "10." ve "/"  ve "." varsa
                eklenecek_deger = j.split("\n")[0] # "DOI" değerlernin "eklenecek_deger" ata
                break  
        sonuc7.append(eklenecek_deger)
    return sonuc7


def make_clickable(url): # make_clickable  altında fonksiyon oluşturma
     return f'<a href="https://doi.org/{url.strip()}" target="_blank">{url.strip()}</a>' # HTML bağlantı linki ve köprü oluşturma

# Anahtar kelime gir
anahtar_kelime = input("Anahtar kelime giriniz: ")

# Web tarayıcısını başlat
driver = webdriver.Chrome()

# Toplamda bir den fazla sayfa  dolaşmak için döngü oluşturma
kataloglar = []
for sayfa_numarasi in range(1,11):
    # URL oluşturma
    url = f"https://dergipark.org.tr/tr/search/{sayfa_numarasi}?q={anahtar_kelime}&section=articles"  
    driver.get(url)

    # Sayfa yüklenene kadar bekleyin
    WebDriverWait(driver, 60).until(EC.invisibility_of_element((By.ID, "loadingOverlay")))

    # Kritere uygun makaleleri listeleyen elementleri bulmak için bekleme yap
    makaleler = WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '[class="card article-card dp-card-outline"]'))
    )

    # Her bir makale için bilgileri ayır
    for makale in makaleler:
        # Makale bilgileri
        katalog = makale.find_element(By.CSS_SELECTOR, '[class="card-body"]').text
        kataloglar.append(katalog)

# Sözlük oluşturma
veri = {
    "Başlık": [baslik_i.split("\n")[0] for baslik_i in kataloglar], # Başlık getirme
    "Kategori": [" " if len(sayfa_i.split("\n")) <= 1 or sayfa_i.split("\n")[1].count(',') > 1 else sayfa_i.split("\n")[1] for sayfa_i in kataloglar], # Kategori getirme
    "Sorumlu Yazarlar": yazar_isimleri(kataloglar), # Yazar isimleri getirme
    "Dergi İsmi": dergi_isimleri(kataloglar), # Dergi isimlerini getirme
    "Cilt Sayısı": cilt_degerlerini_al(kataloglar), # Cilt sayılarını getirme
    "Sayı": sayi_degerlerini_al(kataloglar), # Sayıları getirme
    "Tarih": tarih_bilgisi_ekle(kataloglar), # Tarih bilgilerini getirme
    "Sayfa": sayfa_sayilari_ekle(kataloglar), # Sayfa numarlarını getirme
    "DOI": doi(kataloglar), # DOI getirme
    "Özet" : [",".join(j[3:-1]) for j in [i.split("\n") for i in kataloglar]] # Özet bilgilerini getirme
}

# DataFrame oluşturma
makale_tablosu2 = pd.DataFrame(veri) 

# "make_clickable" kütüphanesindeki URL fonksiyonunu "makale_tablosu2["DOI"]" sütununa uygulama
makale_tablosu2["DOI"] = makale_tablosu2["DOI"].apply(make_clickable)

# "makale_tablosu2" deki "DOI"leri HTML koduna dönüştürme
display(HTML(makale_tablosu2.to_html(escape=False)))

# "makale_tablosu2" yi Excel formatına dönüştürme
makale_tablosu2.to_excel("makale_listesi2.xlsx", index=False)

# Tarayıcıyı kapat
driver.quit()

Anahtar kelime giriniz:  makine öğrenimi


,Başlık,Kategori,Sorumlu Yazarlar,Dergi İsmi,Cilt Sayısı,Sayı,Tarih,Sayfa,DOI,Özet
0,Veri Madenciliği ve Makine Öğrenimi ile Döviz Kuru Tahmini Uygulaması,Araştırma Makalesi,"Oğuz Ata, Ahmet Ersin Erbudak",Fırat Üniversitesi Mühendislik Bilimleri Dergisi,34,2,2022,553 - 563,10.35234/fumbd.1078207,"Makine öğrenimi 1990’lı yıllardan, veri madenciliği kullanılmaya başlandığı döneme dek yaygın,madenciliği ile veri gruplarına göre bölümlere ayrılması sağlanmıştır. Makine öğrenimi ile ilgili,oluşturulmuştur Sayısal halde toplanan veriler 1352 döviz kuru verisi için 4 ayrı makine öğrenimi,strateji ve plan geliştirmektedir Veri madenciliği ve makine öğrenimi metotlarının tahmin programlarının"
1,Makine Öğrenimi Tabanlı Kısa Vadeli Fotovoltaik Çıkış Gücü Tahminlemesi,Araştırma Makalesi,"Berrin Eryılmaz, Heybet Kılıç, Fatih Koçyiğit",EMO Bilimsel Dergi,13,2,2023,57 - 69,,"önemlidir. Bu çalışmanın amacı, literatürde makine öğrenimi modellerinde yaygın olan aşırı öğrenme ve,çıkış gücünü tahmin etmede kullanmak ve bu kapsamda önerilen modeli 10 farklı makine öğrenimi yöntemi,sonucunda GD-RVFL’ nin etkinliği diğer 10 makine öğrenimi modeline göre önemli ölçüde daha iyi performans gösterdiği görülmüştür"
2,Banka Ödemelerinde Dolandırıcılığın Çizge Madenciliği ve Makine Öğrenimi Algoritmalarıyla Tespiti,Araştırma Makalesi,"Hande Çavşi Zaim, Esra Yolaçan, Eyyüp Gülbandılar",Dicle Üniversitesi Mühendislik Fakültesi Mühendislik Dergisi,12,4,2021,615 - 625,10.24012/dumf.1002110,"meydana gelen dolandırıcılık saldırılarıdır. Grafik veri bilimi kullanılması, mevcut analitik ve makine,, degree gibi çizge algoritmaları ile birlikte standart makine öğrenimi yöntemi ile elde edilen,sonuçlar optimize edilmiştir. Bu açıdan çizge madenciliği ve makine öğrenimi algoritmalarının birlikte"
3,Makine Öğrenmesi Yöntemleri İle Eğitim Başarısının Tahmini Modeli,Araştırma Makalesi,"Deniz Zilyas, Atınç Yılmaz",Dicle Üniversitesi Mühendislik Fakültesi Mühendislik Dergisi,14,3,2023,437 - 447,10.24012/dumf.1322273,"Günümüzde makine öğrenmesi yöntemleri etkin bir biçimde kullanılarak pek çok alanda yüksek,da yaygınlaşmaya başlamıştır. Makine öğrenmesi modellerinden elde edilebilecek başarılarla birçok,toplanan veriler kullanılarak; eğitim başarı tahminini yapacak bir makine öğrenmesi modeli ortaya"
4,Diş Hekimliğinde Makine Öğrenimi Uygulamaları,Derleme,"Güler Burcu Senirkentli, Gazi Erkan Bostancı, Mehmet Serdar Güzel, Metehan Unal",Selcuk Dental Journal,9,3,2022,977 - 983,10.15311/selcukdentj.1032041,bazılarını kapsamakta ve diş problemlerinde makine öğreniminin kullanımına ilişkin gelecekteki yönergeleri öngörmektedir.
5,Makine Öğrenimi Sınıflandırma Algoritmalarını Kullanarak Diyabet Tahmini,Konferans Bildirisi,"Shamriz Nahzat, Mete Yağanoğlu",Avrupa Bilim ve Teknoloji Dergisi,,24,2021,53 - 59,10.31590/ejosat.899716,", makine öğreniminin (MÖ) çok çeşitli kullanımları vardır. Makine öğrenimi teknikleri, kanser,sınıflandırıcıyı bulmak için çeşitli makine öğrenimi tekniklerini kullanarak diyabet tahminini yapmak ve bu,) gibi Makine Öğrenimi sınıflandırma yöntemlerini diyabet tahmin etmek için uyguladık. Bu analizde,edebilen bir model göstermektedir. Bu çalışmanın bulgularına göre, diğer makine öğrenimi"
6,Prostat Kanserinin Makine Öğrenimi Yoluyla Değerlendirilmesi,Araştırma Makalesi,"Fatma Söğüt, Evrim Ersin Kangal",International Journal of Pure and Applied Sciences,9,2,2023,274 - 281,10.29132/ijpas.1382974,"Bilgisayarların makine öğrenimi tekniği ile eğitilmesi ile hastaların gereksiz yere zor tetkiklere,maruz kalması engellenebilir. Son yıllarda makine öğrenimi tabanlı hastalık değerlendirme yaklaşımı,kansere bağlı ölüm olan prostat kanserini değerlendirmek için bir makine öğrenimi (ML) yaklaşımı"
7,Hepatit hastalığının tespitinde bulanık mantık ve makine öğrenmesi yöntemlerinin karşılaştırılması,Araştırma Makalesi,"Cengiz Çoşkun, Emre Yüksek",Dicle Üniversitesi Mühendislik Fakültesi Mühendislik Dergisi,14,4,2023,539 - 546,10.24012/dumf.1319102,"seçimi yöntemi uygul